In [ ]:
import cma,torch,os
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 贝叶斯优化，调整参数
best_value = 1000
best_position = None
res_img = None

init_translate = registration.config.init_translate
translate_delta = registration.config.translate_delta
center_point = [init_translate[0]+translate_delta[0]/2, init_translate[1]+translate_delta[1]/2]

# 定义优化参数
initial_guess = np.array([center_point[0], center_point[1], 180])  # 初始猜测解，根据问题维度调整
bounds = np.array((pso.minV, pso.maxV))

space = [
    Real(10, 100, name="sigma0"),
    Integer(100, 256, name="pop_size"),
]

# 定义目标函数，这里只是一个示例函数
def objective_function(x):
    global best_value, best_position, res_img
    x = torch.tensor(x)
    res = registration.similarity(x)
    fitness = -res[0]
    if fitness < best_value:
        best_value = fitness
        best_position = x
        res_img = res[1]
    return fitness

def cma_run(sigma0, pop_size):
    # 初始化 CMA-ES 优化器
    es = cma.CMAEvolutionStrategy(initial_guess, sigma0, {'popsize': pop_size, 
                                                          'bounds':[bounds[0], bounds[1]],
                                                          'maxiter':50,
                                                          'verb_disp': 1,    # 每代打印一次信息
                                                          })

    # 开始优化过程
    while not es.stop():
        # 获取一组新的样本点
        solutions = es.ask()

        # 计算每个样本点的目标函数值
        function_values = [objective_function(solution) for solution in solutions]

        # 将目标函数值反馈给 CMA-ES
        es.tell(solutions, function_values)

        # 输出当前的状态信息（可选）
        es.logger.add()  # write data to logger
    
    # 获取最终的最佳解
    best_solution = es.result.xbest
    best_fitness = es.result.fbest
    return best_fitness


def cma_simulation(sigma, pop_size):
    # 初始化PSO算法
    best_cost = cma_run(sigma, pop_size)
    
    # 执行PSO
    # best_solution, best_cost = pso.run(max_iter=100)

    # 返回PSO运行结果，这里以最优成本作为需要最小化的指标
    return best_cost

@use_named_args(space)
def objective(sigma0, pop_size):
    return cma_simulation(sigma0, pop_size)

# 执行贝叶斯优化
result = gp_minimize(objective, space, n_calls=50, random_state=0)

# 输出最佳参数
print("Best parameters: sigma0 = {:.2f}, pop_size = {:.2f}".format(result.x[0], result.x[1]))
print("Best objective value: {:.4f}".format(result.fun))
